# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [181]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [182]:
# Print current working dir
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Loop for a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join full path to files found using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [183]:
# Initiate a list to store rows of data
full_data_rows_list = [] 
    
# Loop through each file in filepath list
for f in file_path_list:

# Read each csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        
        # Create a csv reader
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extract raw line from csv and add to full_data_rows_list     
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Print total number of rows 
print(len(full_data_rows_list))

# Print a sample of event data rows
print(full_data_rows_list[1])

# Create a smaller event data csv file: event_datafile_full csv used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Manowar', 'Logged In', 'Jacob', 'M', '0', 'Klein', '247.562', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Shell Shock', '200', '1.54354E+12', '73']


In [184]:
# Print number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding

## Now ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code:

#### Creating a Cluster

In [185]:
# Connect to a Cassandra instance
from cassandra.cluster import Cluster
cluster = Cluster()

# Setup a session
session = cluster.connect()

#### Create Keyspace

In [186]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [187]:
# Set KEYSPACE to sparkify
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

#### Now we need to create tables to run the following queries. With Apache Cassandra we'll model the database tables on the queries that we want to run.

## Create queries to ask the following three questions of the data
---





#### Query 1: 
#### Get artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


Create the table for it:

In [188]:
query = "CREATE TABLE IF NOT EXISTS song_session_dtls"
query = query + """(sessionId int, itemInSession int, artist text, song text, length float,
                    PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)    

###### For this query, a composite primary key on ```sessionId``` and ```itemInSession``` were because this would model the required data access pattern. Data is first accessed by Partition of the ```sessionId``` and ```itemInSession``` followed by other columns. Primary key, would create paritioning based on these and allow faster reads on nodes that store this data.

Load data into this table:

In [189]:
# Use the new csv file
file = 'event_datafile_new.csv'

# Read the csv by line
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Define INSERT statement with placeholders
        query = "INSERT INTO song_session_dtls (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        # Get column data in each line
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        
        # Bind values to the CQL INSERT query
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e) 

#### Validate the data model
Run the select query:

In [190]:
query = "SELECT artist, song, length FROM song_session_dtls WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist +",", row.song+",", row.length)

Faithless, Music Matters (Mark Knight Dub), 495.30731201171875


---

#### Query 2: 
#### Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
Create the table for it:

In [191]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + """
                 (userId int, sessionId int, artist text, song text, itemInSession int, firstName text, lastName text,
                 PRIMARY KEY ((userId, sessionId), itemInSession))
                 WITH CLUSTERING ORDER BY (itemInSession ASC)"""
try:
    session.execute(query)
except Exception as e:
    print(e)


###### Similiar to query 1, this query needed to be first accessed via  ```userId``` and ```sessionId``` partitions first and then columns artist, song. Additionally, a clustering column on ```itemInSession``` is added because of sort requirement.

Load data into this table:

In [192]:
# Use the new csv file
file = 'event_datafile_new.csv'

# Read the csv by line
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Define INSERT statement with placeholders
        query = "INSERT INTO song_playlist_session (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        # Get column data in each line
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line

        # Bind values to the CQL INSERT query
        try:
            session.execute(query, (int(userId), int(sessionId), artist, song, int(itemInSession), firstName, lastName))
        except Exception as e:
            print(e) 

#### Validate the data model
Run the select query:

In [193]:
query = "SELECT artist, song, firstName, lastName FROM song_playlist_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist + ",", row.song + ",", row.firstname+ ",", row.lastname)

Down To The Bone, Keep On Keepin' On, Sylvie, Cruz
Three Drives, Greece 2000, Sylvie, Cruz
Sebastien Tellier, Kilometer, Sylvie, Cruz
Lonnie Gordon, Catch You Baby (Steve Pitron & Max Sanna Radio Edit), Sylvie, Cruz


---

#### Query 3: 
#### Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
Create the table for it:

In [194]:
query = "CREATE TABLE IF NOT EXISTS music_app_hst"
query = query + """
                 (song text, firstName text, lastName text, userId int,
                 PRIMARY KEY ((song), userId))"""
try:
    session.execute(query)
except Exception as e:
    print(e)  


###### Finally, for this query, Primary Key is created on on ```song``` to access all columns related to it. Many people share the same first and last names ```user_id``` is more unique so clustering column was added on it.

Load data into this table:

In [195]:
# Read line from csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Define INSERT statement with placeholders
        query = "INSERT INTO music_app_hst (song, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"

        # Get column data in each line
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line

        # Bind values to the CQL INSERT query
        try:
            session.execute(query, (song, firstName, lastName, int(userId)))
        except Exception as e:
            print(e) 

#### Validate the data model
Run the select query:

In [196]:
query = "SELECT firstName, lastName FROM music_app_hst WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [197]:
try:
    session.execute("DROP TABLE song_session_dtls")
    session.execute("DROP TABLE song_playlist_session")
    session.execute("DROP TABLE music_app_hst")
except Exception as e:
    print(e)

### Close the session and cluster connection

In [198]:
session.shutdown()
cluster.shutdown()